In [1]:
import pandas as pd
import numpy as np
import os
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
pd.options.mode.chained_assignment = None  # default='warn'
plt.rcParams['font.family'] = 'Malgun Gothic'
import datetime

In [2]:
def day_modifier(x):
    # x is a number
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x)
        if len(x) == 1:
            return '0' + x
        else:
            return x

In [3]:
def landnum_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        x = str(x).replace('외', '').replace(' ','')
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [4]:
def data_prep(bdtype, tradetype):
    # bdtype is a string: one of 아파트, 연립다세대 or 오피스텔
    # tradetype is a string: one of 매매 or 전월세
    
    basedir = './국토교통부_실거래가_공개시스템/{}/{}/'.format(bdtype, tradetype)
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.csv'))&('(' in f)]
    
    dfs_list = []
    for i, f in tqdm(enumerate(filenames)):
        df = pd.read_csv(basedir + f, encoding='euc-kr', header=15)
        '''if '해제사유발생일' in df.columns.tolist():
            df = df.drop(columns=['해제사유발생일'])'''

        dfs_list.append(df)
    
    concat_df = pd.concat(dfs_list).reset_index(drop=True)
        
    concat_df['번지'] = concat_df['번지'].apply(landnum_modifier)
    
    concat_df['계약년월'] = concat_df['계약년월'].apply(str)
    concat_df['계약일'] = concat_df['계약일'].apply(str)
    
    concat_df['계약날짜기준_건물연식'] = concat_df['계약년월'].apply(lambda x: int(x[:4])) - concat_df['건축년도']
    
    concat_df['계약일'] = concat_df['계약일'].apply(day_modifier)
    
    concat_df['계약날짜'] = concat_df['계약년월'].apply(lambda x: x[:4]) + '-' + concat_df['계약년월'].apply(lambda x: x[-2:])\
    + '-' + concat_df['계약일']
    
    concat_df['계약날짜'] = pd.to_datetime(concat_df['계약날짜'], format='%Y-%m-%d')
    
    concat_df['거래금액(만원)'] = concat_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
    concat_df['전용면적단가(만원/㎡)'] = concat_df['거래금액(만원)'] / concat_df['전용면적(㎡)']
    
    concat_df['지번주소'] = concat_df['시군구'] + ' ' + concat_df['번지']
        
    cols_to_drop = ['시군구', '번지', '본번', '부번', '지번주소', '계약년월', '계약일', '도로명', '해제사유발생일']
        
    concat_df = concat_df[['지번주소', '도로명'] + [col for col in concat_df.columns if col not in cols_to_drop] + ['해제사유발생일']]
    
    concat_df = concat_df.reset_index(drop=True)
    
    return concat_df

In [5]:
yunrip_df = data_prep('연립다세대', '매매')
print(yunrip_df.shape)
yunrip_df.head()

16it [00:01,  9.65it/s]


(688816, 12)


,지번주소,도로명,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일
0,서울특별시 강남구 개포동 1264-3,개포로31길 23-7,(1264-3),53.28,29.23,11500,-1,1992.0,14.0,2006-08-29,215.840841,NaN
1,서울특별시 강남구 개포동 171-13,선릉로14길 11,(171-13),68.08,56.10,56500,2,1988.0,18.0,2006-12-20,829.905993,NaN
2,서울특별시 강남구 개포동 1239-7,개포로15길 25,강남빌라 가동,52.59,45.00,40500,2,1988.0,18.0,2006-12-09,770.108386,NaN
3,서울특별시 강남구 개포동 1239-7,개포로15길 25,강남빌라 가동,52.59,45.00,42000,1,1988.0,18.0,2006-12-19,798.630918,NaN
4,서울특별시 강남구 개포동 1239-6,개포로15길 27,강남빌라 나동,52.75,44.68,22000,2,1988.0,18.0,2006-11-02,417.061611,NaN


In [6]:
apart_df = data_prep('아파트', '매매')
print(apart_df.shape)
apart_df.head()

16it [00:01,  8.74it/s]


(1193184, 11)


,지번주소,도로명,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일
0,서울특별시 강남구 개포동 655-2,언주로 103,개포2차현대아파트(220),77.75,59500,7,1988,18,2006-03-10,765.273312,NaN
1,서울특별시 강남구 개포동 655-2,언주로 103,개포2차현대아파트(220),77.75,60000,6,1988,18,2006-03-29,771.704180,NaN
2,서울특별시 강남구 개포동 655-2,언주로 103,개포2차현대아파트(220),77.75,67000,9,1988,18,2006-04-29,861.736334,NaN
3,서울특별시 강남구 개포동 655-2,언주로 103,개포2차현대아파트(220),77.75,60000,4,1988,18,2006-06-01,771.704180,NaN
4,서울특별시 강남구 개포동 655-2,언주로 103,개포2차현대아파트(220),77.75,72250,5,1988,18,2006-10-20,929.260450,NaN


In [7]:
officetel_df = data_prep('오피스텔', '매매')
print(officetel_df.shape)
officetel_df.head()

16it [00:00, 49.52it/s]


(178580, 11)


,지번주소,도로명,단지명,전용면적(㎡),거래금액(만원),층,건축년도,계약날짜기준_건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일
0,서울특별시 강남구 개포동 13-3,개포로 623,대청타워,32.44,6400,8,1997.0,9.0,2006-01-10,197.287300,NaN
1,서울특별시 강남구 개포동 13-3,개포로 623,대청타워,32.44,7100,7,1997.0,9.0,2006-01-10,218.865598,NaN
2,서울특별시 강남구 개포동 13-3,개포로 623,대청타워,32.44,7300,11,1997.0,9.0,2006-01-11,225.030826,NaN
3,서울특별시 강남구 개포동 13-3,개포로 623,대청타워,31.91,7500,10,1997.0,9.0,2006-01-18,235.036039,NaN
4,서울특별시 강남구 개포동 13-3,개포로 623,대청타워,32.44,7400,15,1997.0,9.0,2006-01-19,228.113440,NaN


In [8]:
yunrip_df['부동산유형'] = '연립다세대'
apart_df['부동산유형'] = '아파트'
officetel_df['부동산유형'] = '오피스텔'

In [9]:
bulktrade_basedir = './국토교통부_실거래가_공개시스템/집값분석/일괄구매/'

In [16]:
yunrip_selected_df = pd.read_csv(bulktrade_basedir + '연립다세대_일괄구매_전체.csv', index_col=[0])
print(yunrip_selected_df.shape)
yunrip_selected_df.head()

(56781, 12)


,지번주소,건물명,전용면적(㎡),대지권면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일,부동산유형
224,서울특별시 강남구 개포동 1221-10,셍상스1,27.34,17.75,9940,3,1996.0,10.0,2006-11-09,363.569861,NaN,연립다세대
225,서울특별시 강남구 개포동 1221-10,셍상스1,27.34,17.75,9940,2,1996.0,10.0,2006-11-09,363.569861,NaN,연립다세대
226,서울특별시 강남구 개포동 1221-10,셍상스1,27.34,17.75,9940,3,1996.0,10.0,2006-11-09,363.569861,NaN,연립다세대
227,서울특별시 강남구 개포동 1221-10,셍상스1,37.75,24.48,13720,4,1996.0,10.0,2006-11-09,363.443709,NaN,연립다세대
228,서울특별시 강남구 개포동 1221-10,셍상스1,39.17,25.40,14240,3,1996.0,10.0,2006-11-09,363.543528,NaN,연립다세대


In [17]:
apart_selected_df = pd.read_csv(bulktrade_basedir + '아파트_일괄구매_전체.csv', index_col=[0])
print(apart_selected_df.shape)
apart_selected_df.head()

(9000, 11)


,지번주소,단지명,전용면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일,부동산유형
835105,서울특별시 강남구 개포동 141-0,개포주공 1단지,50.64,140000,5,1982,35,2017-07-11,2764.612954,NaN,아파트
835106,서울특별시 강남구 개포동 141-0,개포주공 1단지,50.64,133000,4,1982,35,2017-07-11,2626.382306,NaN,아파트
835107,서울특별시 강남구 개포동 141-0,개포주공 1단지,50.64,138000,3,1982,35,2017-07-11,2725.118483,NaN,아파트
835108,서울특별시 강남구 개포동 141-0,개포주공 1단지,41.98,120000,4,1982,35,2017-07-11,2858.504050,NaN,아파트
835109,서울특별시 강남구 개포동 141-0,개포주공 1단지,42.55,120000,4,1982,35,2017-07-11,2820.211516,NaN,아파트


In [18]:
officetel_selected_df = pd.read_csv(bulktrade_basedir + '오피스텔_일괄구매_전체.csv', index_col=[0])
print(officetel_selected_df.shape)
officetel_selected_df.head()

(15155, 11)


,지번주소,단지명,전용면적(㎡),거래금액(만원),층,건축년도,건물연식,계약날짜,전용면적단가(만원/㎡),해제사유발생일,부동산유형
39487,서울특별시 강남구 논현동 241-3,강남구청역 에스케이 허브 블루,79.22,41486,15,2005.0,4.0,2009-11-20,523.680889,NaN,오피스텔
39488,서울특별시 강남구 논현동 241-3,강남구청역 에스케이 허브 블루,79.22,37272,7,2005.0,4.0,2009-11-20,470.487251,NaN,오피스텔
39489,서울특별시 강남구 논현동 241-3,강남구청역 에스케이 허브 블루,78.05,40831,14,2005.0,4.0,2009-11-20,523.139013,NaN,오피스텔
39490,서울특별시 강남구 논현동 241-3,강남구청역 에스케이 허브 블루,78.05,40831,13,2005.0,4.0,2009-11-20,523.139013,NaN,오피스텔
39491,서울특별시 강남구 논현동 241-3,강남구청역 에스케이 허브 블루,79.22,37272,6,2005.0,4.0,2009-11-20,470.487251,NaN,오피스텔


In [19]:
yunrip_df.loc[yunrip_selected_df.index, '거래금액(만원)'] = yunrip_df.loc[yunrip_selected_df.index, '거래금액(만원)'] / 0.9
apart_df.loc[apart_selected_df.index, '거래금액(만원)'] = apart_df.loc[apart_selected_df.index, '거래금액(만원)'] / 0.9
officetel_df.loc[officetel_selected_df.index, '거래금액(만원)'] = officetel_df.loc[officetel_selected_df.index, '거래금액(만원)'] / 0.9

In [20]:
yunrip_df.drop(columns=['전용면적단가(만원/㎡)'], inplace=True)
apart_df.drop(columns=['전용면적단가(만원/㎡)'], inplace=True)
officetel_df.drop(columns=['전용면적단가(만원/㎡)'], inplace=True)

In [22]:
%%time
yunrip_df.to_csv('./prepped_data/연립다세대_메인_20060101_20210409.csv')
apart_df.to_csv('./prepped_data/아파트_메인_20060101_20210409.csv')
officetel_df.to_csv('./prepped_data/오피스텔_메인_20060101_20210409.csv')

Wall time: 18.8 s
